In [ ]:
# PyTorch libraries and modules
import random
from torchvision.transforms import functional as TF

class ShiftWithPeriodicBoundary(object):
    """Shift the image with periodic boundary conditions."""
    def __init__(self, max_shift=5):
        self.max_shift = max_shift  # Maximum pixels to shift

    def __call__(self, img):
        # Determine the amount and direction of the shift
        shift_x = random.randint(-self.max_shift, self.max_shift)
        shift_y = random.randint(-self.max_shift, self.max_shift)

        # Convert image to tensor to use torch functions, then shift
        img_tensor = TF.to_tensor(img)
        img_shifted = torch.roll(img_tensor, shifts=(shift_x, shift_y), dims=(1, 2))

        # Convert back to PIL Image to be compatible with other transforms
        img_shifted = TF.to_pil_image(img_shifted)
        return img_shifted

# Update the transform to include the shift with periodic boundary conditions
transform_with_shift = transforms.Compose([
    ShiftWithPeriodicBoundary(5),  # Apply our custom transform with a max shift of 5 pixels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Reinitialize datasets with the new transform
trainset_shifted = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_with_shift)
trainloader_shifted = torch.utils.data.DataLoader(trainset_shifted, batch_size=64, shuffle=True)

# Now, you can use trainloader_shifted for training your model to test the new transformation.
# Below is an example on how to visualize one shifted image.

import matplotlib.pyplot as plt

# Get a batch of training data
images, _ = next(iter(trainloader_shifted))

# Show one of the images
plt.figure(figsize=(2,2))
plt.imshow(images[0].squeeze(), cmap='gray')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from PIL import Image

# Load and prepare the image
image_path = 'your_image.png'  # Replace 'your_image.png' with your image file
image = Image.open(image_path)
image = image.resize((360, 180))  # Resize to fit the torus (adjust as needed)
image = np.array(image) / 255.0  # Normalize pixel values

# Torus parameters
R = 2  # Major radius
r = 1  # Minor radius

# Create meshgrid for torus angles theta (θ) and phi (φ)
theta = np.linspace(0, 2 * np.pi, image.shape[1])
phi = np.linspace(0, 2 * np.pi, image.shape[0])
theta, phi = np.meshgrid(theta, phi)

# Torus parameterization to Cartesian coordinates
x = (R + r * np.cos(phi)) * np.cos(theta)
y = (R + r * np.cos(phi)) * np.sin(theta)
z = r * np.sin(phi)

# Create 3D plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Map the image onto the torus
# Note: We use the facecolors argument to set the color of each "pixel" on the torus
ax.plot_surface(x, y, z, rstride=5, cstride=5, facecolors=image, shade=False)

# Hide grid and axes for better visualization
ax.grid(False)
ax.axis('off')

plt.show()
